In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
#tensorflow as machine learning frameworks
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
import os

In [4]:
# Define your data directories
#'path_to_your_data'
data_dir = 'imageData'

train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')

In [5]:
# Define your batch size
batch_size = 32

# Define your image size
img_height = 150
img_width = 150

In [6]:
# Define your data generators with default parameter 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 14034 images belonging to 6 classes.
Found 0 images belonging to 0 classes.
Found 3000 images belonging to 6 classes.


In [7]:
# Basic model with 3 convelational layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(6, activation='softmax')  # 6 classes for 0,1,2,3,4,5
])

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(train_generator, epochs=1, validation_data=validation_generator)
test_loss, test_acc = model.evaluate(test_generator)

94/94 [==============================] - 21s 224ms/step - loss: 0.8857 - accuracy: 0.6767


In [12]:
# Saveing
model.save('my_model.h5')

In [13]:
# Save the model weights
model.save_weights('my_model_weights.h5')

# Compare my model with pre-trained models, Xception and ResNet50.

In [14]:
#Xception pretraind Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [16]:
# Load the pre-trained Xception model
xception = Xception(weights='imagenet', include_top=False)

# Add a global average pooling layer and a dense output layer
x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add a dense hidden layer (optional)
predictions = Dense(6, activation='softmax')(x)  # 6 classes for 0,1,2,3,4,5

# Define the new model
xception_model = Model(inputs=xception.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in xception_model.layers[:-3]:
    layer.trainable = False

# Compile the model
xception_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
xception_model.fit(train_generator, epochs=1, validation_data=validation_generator)

# Evaluate the model
xception_test_loss, xception_test_acc = xception_model.evaluate(test_generator)

94/94 [==============================] - 148s 2s/step - loss: 0.3154 - accuracy: 0.8833


In [17]:
# Load the pre-trained ResNet50 model
from tensorflow.keras.applications import ResNet50


In [18]:
# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False)

# Add a global average pooling layer and a dense output layer
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add a dense hidden layer (optional)
predictions = Dense(6, activation='softmax')(x)  # 6 classes for 0,1,2,3,4,5

# Define the new model
resnet_model = Model(inputs=resnet.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in resnet_model.layers[:-3]:
    layer.trainable = False

# Compile the model
resnet_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet_model.fit(train_generator, epochs=1, validation_data=validation_generator)

# Evaluate the model
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_generator)

94/94 [==============================] - 141s 1s/step - loss: 1.3198 - accuracy: 0.4220


# Inference on a new image using trained models

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
img_path = '52.jpg'
#img_path = 'test.jpg'
img = image.load_img(img_path, target_size=(150, 150))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Scale pixel values to the range [0, 1]

# Make a prediction using the first model
predictions1 = model.predict(img_array)

# Make a prediction using the second model
predictions2 = xception_model.predict(img_array)

# Make a prediction using the third model
predictions3 = resnet_model.predict(img_array)

# Print the predictions
print('Model 1 predictions:', predictions1)
print('Model 2 predictions:', predictions2)
print('Model 3 predictions:', predictions3)
print("Category:: 0 for buildings, 1 for forest, 2 for glacier, 3 formountain, 4 for sea, and 5 for street")

1/1 [==============================] - 0s 77ms/step
Model 1 predictions: [[1.1199812e-02 2.1843417e-03 8.4777065e-03 9.6888876e-01 8.5010594e-03
  7.4834685e-04]]
Model 2 predictions: [[8.4445765e-06 1.5274829e-05 1.5310109e-01 8.4686631e-01 6.4953374e-06
  2.3556297e-06]]
Model 3 predictions: [[0.06547568 0.08508906 0.29942095 0.17955221 0.25603646 0.11442565]]
Category:: 0 for buildings, 1 for forest, 2 for glacier, 3 formountain, 4 for sea, and 5 for street


In [23]:
# Get the index of the predicted class for the first model
predicted_class1 = np.argmax(predictions1)

# Get the index of the predicted class for the second model
predicted_class2 = np.argmax(predictions2)

# Get the index of the predicted class for the third model
predicted_class3 = np.argmax(predictions3)

# Print the predicted classes
print('model 1 predicted class:', predicted_class1)
print('xception_model predicted class:', predicted_class2)
print('resnet_model predicted class:', predicted_class3)
print("Category:: 0 for buildings, 1 for forest, 2 for glacier, 3 formountain, 4 for sea, and 5 for street")

model 1 predicted class: 3
xception_model predicted class: 3
resnet_model predicted class: 2
Category:: 0 for buildings, 1 for forest, 2 for glacier, 3 formountain, 4 for sea, and 5 for street
